# basics

In [1]:
e=new.env()

In [2]:
e$a = 1

In [3]:
e$a <- 1


以下はエラーになる

In [10]:
eval(a , envir = e)

ERROR: Error in eval(a, envir = e):  オブジェクト 'a' がありません 


以下はOK:

In [11]:
e$a

[1] 1

environmentのリストを表示：

In [5]:
search()


[1] ".GlobalEnv"        "jupyter:irkernel"  "package:stats"    
 [4] "package:graphics"  "package:grDevices" "package:utils"    
 [7] "package:datasets"  "package:methods"   "Autoloads"        
[10] "package:base"

In [12]:
ls.str(e)

a :  num 1

In [13]:
ls(e)

[1] "a"

## extracting values

### 環境eに限定する取り出し方

In [14]:
e$a

[1] 1

In [15]:
e[["a"]]

[1] 1

### 必要に応じて遡る取り出し方

In [17]:
get("a" , envir = e)

[1] 1

In [18]:
?get

# 環境からのオブジェクトの削除
list の場合はNULLを代入すればよいが、環境の場合はrmを使う

In [19]:
e$b = 1

In [20]:
ls(e)

[1] "a" "b"

In [21]:
rm("b" , envir = e)

In [22]:
ls(e)

[1] "a"

## 存在確認

In [23]:
x = 1

必要に応じて遡るバージョン

In [25]:
exists("x" , envir = e)

[1] TRUE

遡らないバージョン：

In [26]:
exists("x" , envir = e , inherits = F)

[1] FALSE

## 環境の比較


In [28]:
identical(e , globalenv())

[1] FALSE

In [29]:
identical(environment() , globalenv())

[1] TRUE

# recursing over environments

In [30]:
library(pryr)

Warning message:
"package 'pryr' was built under R version 3.4.3"

In [31]:
where("x")

<environment: R_GlobalEnv>

In [34]:
where("a"  , env = e)

<environment: 0x0000000019cec678>

In [33]:
?where

# function環境
- enclosing　：　関数が作られた環境（これのみ１つに限定）
- binding : 関数が定義されている環境（関数を作った時に違う環境に定義を押し込むことがあるため、enclosingとbindingは異なり得る）
- execution　：　関数実行中に（実行に伴って作られる変数を格納するために）短時間だけ作られる環境
- calling : 関数が呼び出された環境

## environment()はenclosing environmentを返す

In [35]:
f = function(x) { x + y}

In [37]:
environment(f)

<environment: R_GlobalEnv>

In [ ]:
y = 1

In [40]:
where("f")

<environment: R_GlobalEnv>

## pryrのwhereはbinding environmentを返す（？）

In [42]:
assign("g" , f , envir = e)

In [45]:
where("g" , env = e)

<environment: 0x0000000019cec678>

## enclosingとbindingの関係

- 関数を呼び出すとき、binding環境のなかで名前の解決が行われる。
- 名前が解決されるとその関数がどこでもともと定義されたか（＝enclosing環境）がわかるので、関数の内容をそこに読みに行く

In [46]:
environment(sd)

<environment: namespace:stats>

In [47]:
where("sd")

<environment: package:stats>
attr(,"name")
[1] "package:stats"
attr(,"path")
[1] "C:/Program Files/R/R-3.4.1/library/stats"

- package環境はpublicアクセス可能な環境であり、サーチパス上にある
- namespace環境はprivateな関数なども含む環境

## execution environments

fresh start原則により、execution環境でaが一回定義されても、もう一回gを走らせたときには違うexecution環境が作られるので２回目にgを走らせたときも１回目とおなじことが起こる:

In [48]:
g <- function(x) {
  if (!exists("a", inherits = FALSE)) {
    message("Defining a")
    a <- 1
  } else {
    a <- a + 1
  }
  a
}

In [49]:
g(1)

Defining a


[1] 1

In [50]:
g(1)

Defining a


[1] 1

関数のなかで関数を定義した場合：
入れ子関数のenclosing環境＝外側の関数の実行環境

In [52]:
plus <- function(x) {
  function(y) x + y
}

In [ ]:
plus_one <- plus(1)

In [53]:
identical(parent.env(environment(plus_one)), environment(plus))

[1] TRUE

上式の意味：
- plus_oneのbinding environmentがplusの実行環境であることを確かめたい
- plus_oneのbinding environmentの親がplusのbinding environmentであることはその間接的なチェックとなっている

より直接的なチェック：

In [58]:
plus2 = function(x) {
    e = environment()
    run_inloop = function(y){ x + y }
    return(list(e , run_inloop))
    
}

In [61]:
run_return = plus2(2)

In [64]:
identical(environment(run_return[[2]]) , run_return[[1]])

[1] TRUE

## calling環境

関数の内部でのオブジェクトの探索順序は
- enclosing environment
- 実行環境（<==こちらを見に行くことをdynamic scopingという）
    - dynamic scopingを行う言語はあまりない（emacs lispは例外）

の順番である：


In [68]:
h <- function() {
  x <- 10
  function() {
    x
  }
}
i <- h()
x <- 20
i()

[1] 10

In [72]:
h2 <- function() {
  
  function() {
    xx
  }
}
i <- h2()
xx <- 20
i()

ERROR: Error in i():  オブジェクト 'xx' がありません 


In [74]:
rm("xx")

Warning message in rm("xx"):
" オブジェクト 'xx' がありません "

### calling環境にはparent.frame()でアクセス可能

In [73]:
f2 <- function() {
  x <- 10
  function() {
    def <- get("x", environment())
    cll <- get("x", parent.frame())
    list(defined = def, called = cll)
  }
}
g2 <- f2()
x <- 20
str(g2())

List of 2
 $ defined: num 10
 $ called : num 20


# binding names to values

## <<-は環境を遡っていって見つかたオブジェクトに対して操作を行う


- 存在しなければglobal環境にオブジェクトを作成する
    - あまり望ましくない
- <<-はclosure関連で使われることが典型的

## 遅延bindings

- 参照時まで遅延させる
- 本来はdelayedAssign()を用いる
- pryrを使った場合は以下の通り

In [78]:
x3 %<d-% 3

In [76]:
x3

[1] 3

In [77]:
rm("x3")

In [80]:
ls()

[1] "e"          "f"          "f2"         "g"          "g2"        
 [6] "h"          "h2"         "i"          "plus"       "plus_one"  
[11] "plus_two"   "plus2"      "run_return" "x"          "x3"        
[16] "y"

## active binding

- 参照されるたびに評価する
- 本来はmakeActiveBindings()を使う
- pryrでは以下のようにする

In [81]:
x4 %<a-% runif(1)

In [82]:
x4

[1] 0.08960548

In [83]:
x4

[1] 0.1684105

# Explicit environments

In [88]:
modify <- function(x) {
  x$a <- 2
  #invisible()
}

listに適用すると値渡しなのでx_lは変化しない：

In [89]:
x_l <- list()
x_l$a <- 1
modify(x_l)
x_l$a

[1] 1

他方、環境に適用するとx_eが変化することが確認できる：

In [90]:
x_e <- new.env()
x_e$a <- 1
modify(x_e)
x_e$a

[1] 2

## まっさらな環境をつくりたいときはparentをempty環境にするようにするべき：

そうでないと以下のようなことが起きる：

In [92]:
x <- 1
e1 <- new.env()
get("x", envir = e1)

[1] 1

まっさらな環境を作りたいのであれば以下のようにエラーが返ってきてほしい：

In [91]:
e2 <- new.env(parent = emptyenv())
get("x", envir = e2)

ERROR: Error in get("x", envir = e2):  オブジェクト 'x' がありません 


## コピーを避ける

- 環境はコピーされないので、listよりもbig dataハンドリング上都合がよい場合があった
- R 3.1.0からlistもdeep copyを避けるようになった

## パッケージstate

In [93]:
my_env <- new.env(parent = emptyenv())
my_env$a <- 1

get_a <- function() {
  my_env$a
}
set_a <- function(value) {
  old <- my_env$a
  my_env$a <- value
  invisible(old)
}

返り値はoldだが、invisible指定されているので 

なんとか= set_a(なんとか）

としたときの動作はinvisibleがない場合と同一だが、以下のような呼び方だと画面にoldが表示されない：

In [94]:
set_a(3)

In [99]:
get_a()

[1] 6

In [100]:
set_a(2)

In [101]:
old_value = set_a(6)

In [102]:
old_value

[1] 2

old valueを返すようにすると便利、といったことが書いてあるがとりあえずスキップしよう・・・